In [2]:
# from utils.segmentation import YOLOSegmentation
import cv2
import numpy as np
import rospy
from cv_bridge import CvBridge, CvBridgeError
from contact_grasp.srv import segmentationSrv, segmentationSrvResponse
from utils.camera_utils import RealCamera, RealCameraROS
# from utils.visualisation_utils import depth2pc
from utils.transform_utils import regularize_pc, depth2pc
from utils.ROS_utils import generate_grasps_client, format_pointcloud_msg, run_action
import open3d as o3d
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-07-04 23:00:07,925 - utils - NumExpr defaulting to 8 threads.


In [3]:
def display_inlier_outliers(cloud, ind):
    print("1")
    inlier_cloud = cloud.select_by_index(ind)
    print("2")
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [4]:
camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")


Camera topic found


In [5]:
rgb, depth_image, depth_scale = camera.get_rgb_depth()
depth = depth_image * depth_scale

bridge = CvBridge()

pc_fused, pc_colors_fused = depth2pc(depth, intrinsic, rgb)

In [6]:
bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
cv2.imshow("bgr", bgr)
cv2.imshow("depth", depth)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# cv2.imshow("rgb", rgb)
# cv2.imshow("depth", depth)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
pc_fused, pc_colors_fused = regularize_pc(pc_fused, pc_colors_fused, downsampling_method="voxel", voxel_size=0.005, outlier_filtering_method=None, radius_param_arg=[25, 0.015])
# pc_fused_, pc_colors_fused_ = regularize_pc(pc_fused, pc_colors_fused, downsampling_method="voxel", voxel_size=0.005, outlier_filtering_method="statistical", statistical_param_arg=[20, 2.0])

Downsampling time: 1.4406025409698486s, Filtering time: 2.384185791015625e-07s


In [9]:
pc_fused, pc_colors_fused = regularize_pc(pc_fused, pc_colors_fused, 
                                                  downsampling_method="voxel", voxel_size=0.005,
                                                  outlier_filtering_method=None)

Downsampling time: 0.04045414924621582s, Filtering time: 7.152557373046875e-07s


In [25]:
import open3d as o3d
pc_o3d = o3d.geometry.PointCloud()
pc_o3d.points = o3d.utility.Vector3dVector(pc_fused)
pc_o3d.colors = o3d.utility.Vector3dVector(pc_colors_fused/255)
coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
coordinate.translate([0,0 , 0.])
o3d.visualization.draw_geometries([coordinate, pc_o3d])

In [24]:
# Radius outlier removal:
cl_rad, ind_rad = pc_o3d.remove_radius_outlier(nb_points=25, radius=0.015)
outlier_rad_cl = pc_o3d.select_by_index(ind_rad, invert=True)
outlier_rad_cl.paint_uniform_color([1., 0., 1.0])

# Statistical outlier removal:
cl_stat, ind_stat = pc_o3d.remove_statistical_outlier(nb_neighbors=5,
                                                std_ratio=1.0)
outlier_stat_cl = pc_o3d.select_by_index(ind_stat, invert=True)
outlier_stat_cl.paint_uniform_color([0., 0., 1.])

# Translate to visualize:
points = np.asarray(cl_stat.points)
points += [1.5, 0., 0]
cl_stat.points = o3d.utility.Vector3dVector(points)

points = np.asarray(outlier_stat_cl.points)
points += [1.5, 0., 0]
outlier_stat_cl.points = o3d.utility.Vector3dVector(points)

# Display:
o3d.visualization.draw_geometries([cl_stat, cl_rad, outlier_stat_cl, outlier_rad_cl])

In [11]:
pc2_msg = format_pointcloud_msg(pc_fused, pc_colors_fused)
bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
bgr_msg = bridge.cv2_to_imgmsg(rgb, encoding="rgb8")

In [28]:
orn, pos, opening, score, detected, detected_with_collision = generate_grasps_client(pc2_msg, bgr_msg)



calling service
